# AHN Preprocessing
---
A notebook for preprocessing AHN data to ground surfaces.
- _[AHN](https://www.ahn.nl) data_ (publicly available)

#### 0. Import Modules

In [ ]:
# Add project src to path.
import set_path

# Import modules.
from tqdm import tqdm
import pathlib
import numpy as np
import open3d as o3d
import utils.las_utils as las_utils
import utils.ahn_utils as ahn_utils
import utils.math_utils as math_utils
import utils.o3d_utils as o3d_utils
from utils.interpolation import FastGridInterpolator
import preprocessing.ahn_preprocessing as ahn_preprocessing

#### 1. Preprocess

In [ ]:
ahn_cloud_folder = '../../../datasets/AHN/'
tree_dataset_folder = '../../../datasets/dataset_1/'

In [ ]:
# collect ahn files
ahn_data_folder = pathlib.Path(ahn_cloud_folder)
ahn_bboxes = las_utils.ahn_bbox_las_folder(ahn_data_folder)
print(f"- {len(ahn_bboxes)} AHN clouds: {list(ahn_bboxes.keys())}")

# collect treecodes
data_folder = pathlib.Path(tree_dataset_folder)
surface_out_folder = data_folder.joinpath('ahn_surf/')
file_types = ('.LAS', '.las', '.LAZ', '.laz')
treecodes = set([las_utils.get_treecode_from_filename(f.name) for f in data_folder.glob('*/filtered_tree_*')
             if f.name.endswith(file_types)])
print(f"- {len(treecodes)} treecodes")

treecodes_ahn = {key: [] for key in ahn_bboxes.keys()}
for treecode in treecodes:
    bbox = las_utils.get_bbox_treecode(treecode, 10)
    for ahn_file, ahn_bbox in ahn_bboxes.items():
        if math_utils.bbox_in_bbox(bbox, ahn_bbox):
            treecodes_ahn[ahn_file].append(treecode)
            break

In [ ]:
# Preprocess all treecodes
# WARNING: this may require 16+ GB of RAM, depending on the size of the clouda.
# WARNING: this may significant amount of time.
for ahn_file, treecodes in treecodes_ahn.items():
    print(ahn_file)
    if len(treecodes) > 0:
        ahn_cloud = las_utils.read_las(ahn_data_folder.joinpath(ahn_file))
        ahn_preprocessing.clip_ahn_treecodes(ahn_cloud, treecodes, surface_out_folder, buffer=10)

**Example:** elevation level of a tree

In [ ]:
tree_cloud = o3d_utils.read_las('../../../datasets/dataset_1/cyclomedia/filtered_tree_119305_485108.laz')
ground_cloud = o3d_utils.read_las('../../../datasets/dataset_1/ahn_surf/ahn_surf_119305_485108.laz')

npz_reader = ahn_utils.NPZReader('../../../datasets/dataset_1/ahn_surf/')
npz_reader.tree_surface_z('119305_485108')

In [ ]:
o3d.visualization.draw_geometries([ground_cloud, tree_cloud])